# Datenbezug über SRU und Umwandlung der Inhalte in eine Tabelle:

## Grundlagen zur SRU-Schnittstelle der DNB:

Die DNB bietet Ihre Daten auf drei verschiedene "Kataloge" gesplittet an, von denen immer einer für eine Abfrage ausgewählt werden muss. Dies geschieht über eine Erweiterung der Base-URL. Zur Verfügung stehen folgende Kataloge:

- Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
- Katalog des Deutschen Musikarchivs (DMA) - Datensätze des Deutschen Musikarchivs
- Katalog der Gemeinsamen Normdatei (GND) - hierin befinden sich die Normdaten

Die erweiterungen für die URL sind folgende:

- DNB: https://services.dnb.de/sru/dnb
- DMA: https://services.dnb.de/sru/dnb.dma
- GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine "Explain-Response" in XML zurück. 
Mit Hilfe der Bibliothek BeautifulSoup kann die Antwort direkt in XML umgewandelt werden.

[Weitere Informationen und Dokumentation zur SRU-Schnittstelle](https://www.dnb.de/sru)

### Laden benötigter Zusatzbibliotheken: 

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


## SRU-Anfrage

In [ ]:
#Base-URL für den  Katalog der DNB (Titeldaten):
dnb = requests.get("https://services.dnb.de/sru/dnb")

#Einfache Amfrage an die Schnittstelle ohne Suche 
response = soup(dnb.content, features="xml")  #Parsen der Schnittstellenantwort als xml
print(response.prettify()[0:500])  #Einschränken der Ausgabe auf die ersten 500 Zeichen

Für eine Suchanfrage an die Daten der DNB wird nun zunächst über die Wahl der URL der Katalog definiert. Mit Hilfe der Variable parameter werdem dann alle weiteren Parameter, die die SRU-Schnittstelle benötigt, übergeben.

In [ ]:
#Base-URL:
dnb_url = "https://services.dnb.de/sru/dnb"

#Parameter, die mit der Anfrage übergeben werden sollen: 
parameter = {'version' : '1.1' , 
             'operation' : 'searchRetrieve' , 
             'query' : 'Klimawandel and jhr=2010', 
             'recordSchema' : 'MARC21-xml'} 

#Abfrage der Schnittstelle:
r = requests.get(dnb_url, params = parameter)
#Parsen der Antwort in xml:
response = soup(r.content, features="xml")
#Überführen der einzelnen, in der Antwort enthaltenen Datensätze/Treffer ('Records') in eine Liste:
result = response.find_all('record', {'type':'Bibliographic'})
print(response.prettify()[0:750])

In [ ]:
number = response.find('numberOfRecords')
print(number.text, 'Ergebnisse')
print(len(result))

Wichtig: Die SRU-Schnittstelle liefert per default zunächst immer nur 10 Datensätze auf einmal aus, maximal 100.  

### Umwandlung in eine Funktion:

Funktion, die dank while-Schleife alle Datensätze der Anfrage sammelt:  

In [ ]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    i = 0 
    records = []

    while i % 100 == 0: 
        params.update({'startRecord': i})       
        r = requests.get(base_url, params=params)
        xml = soup(r.content, features="xml")
        result = xml.find_all('record', {'type':'Bibliographic'})
        records+=result
        num_results = len(result)
        if num_results != 100:
            break
        else: 
            i += 100  
        
    return records

In [ ]:
records = dnb_sru('Studentenbewegung')  #and location=onlinefree and cod=pdf')
print(len(records), 'Ergebnisse')

In [ ]:
print(records[137].prettify())

## Inhalte aus dem XML extrahieren: 

### Einblick: 

In [ ]:
test_record = records[137]
test_record = str(test_record)
print(test_record)

In [ ]:
# Parsen des XML: 
root = etree.fromstring(test_record)
print(root)

In [ ]:
# Extrahieren des Inhalts des Controlfields "001"
controlfield_001 = root.find(".//controlfield[@tag='001']")
print(controlfield_001)

In [ ]:
# Definieren des Namespaces: 
namespace = {'marc': 'http://www.loc.gov/MARC21/slim'}

In [ ]:
controlfield_001 = root.find('.//marc:controlfield[@tag="001"]', namespaces=namespace)#.text
print(controlfield_001)

In [ ]:
datafield_245_a = root.xpath('.//marc:datafield[@tag="245"]/marc:subfield[@code="a"]/text()', namespaces=namespace)
print(datafield_245_a)

In [ ]:
print(type(datafield_245_a))

#### Erstellen einer einfachen Funktion: 

In [ ]:
def simple_parse_record(record):

    record = str(record).replace('\x98', '').replace('\x9c', '')
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
        
    meta_dict = {"idn":idn, "titel":titel}
    #print(meta_dict)
    
    return meta_dict

In [ ]:
output = [simple_parse_record(record) for record in records]

In [ ]:
df_simple = pd.DataFrame(output)
df_simple

### Schönere, besser zu bedienende Funktion: 

In [ ]:
def parse_record(record):
    
    record = str(record).replace('\x98', '').replace('\x9c', '')
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    def extract_text(xpath_query):
        fields = xml.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text for field in fields if field.text)
        return "unknown"

    
    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    author_gnd = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='0']")
    links = extract_text("marc:datafield[@tag='856']/marc:subfield[@code='u']")
    
    #add_author = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    #add_author_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")

    
    return {"idn": idn, "titel": titel, "author": author, "rela":author_rela, "gnd":author_gnd} #, "links":links}
                                    #"additional author":add_author, "added_rela": add_author_rela}

In [ ]:
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

## Speichern: 

In [ ]:
df.to_csv("Studentenbewegung.csv", index=False, encoding="utf-8")